In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de ejemplo (puedes cambiarla según lo que busques)
url = "https://www.recambioscoches.es/categoria/frenos"

# Simulamos un navegador con User-Agent
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Descargar la página
response = requests.get(url, headers = headers)

# Verificar si la petición fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Buscar los elementos de los productos (ajusta las clases según la web)
    products = soup.find_all("div", class_ = "product-list-item")

    # Extraer datos
    data = []
    for product in products:
        name = product.find("a", class_ = "product-name").text.strip()
        price = product.find("span", class_ = "price").text.strip()
        data.append({"Nombre": name, "Precio": price})

    # Guardar en un DataFrame
    df = pd.DataFrame(data)
    print(df)

else:
    print("Error al acceder a la web")

In [ ]:
df.to_csv("repuestos.csv", index = False, encoding = "utf-8")
print("data_repuestos.csv")

---------------

In [2]:
import requests
import pandas as pd

def get_ovoko_parts(marca = "toyota", paginas = 5):
    datos = []

    for pagina in range(1, paginas + 1):
        url = f"https://api.ovoko.eu/es/v1/products/search?query={marca}&page={pagina}&limit=24"

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept": "application/json",
        }

        respuesta = requests.get(url, headers = headers)
        
        if respuesta.status_code == 200:
            resultados = respuesta.json()
            productos = resultados.get("products", [])

            for prod in productos:
                datos.append({
                    "Nombre": prod.get("title"),
                    "Precio (€)": prod.get("price", {}).get("amount") / 100 if prod.get("price") else None,
                    "Moneda": prod.get("price", {}).get("currency", ""),
                    "Marca": marca,
                    "Ubicación": prod.get("location", {}).get("country", ""),
                    "Estado": prod.get("condition", {}).get("label", ""),
                    "Referencia OEM": prod.get("oem_numbers", [None])[0]
                })
        else:
            print(f"Error en la página {pagina}: código {respuesta.status_code}")

    return pd.DataFrame(datos)

# Obtener datos de Toyota y Lexus
df_toyota = get_ovoko_parts("toyota", paginas = 10)
df_lexus = get_ovoko_parts("lexus", paginas = 10)

# Unir ambos resultados
df_total = pd.concat([df_toyota, df_lexus], ignore_index = True)

# Guardar en CSV
df_total.to_csv("repuestos_toyota_lexus.csv", index = False, encoding = "utf-8")

print(df_total.head())

Error en la página 1: código 403
Error en la página 2: código 403
Error en la página 3: código 403
Error en la página 4: código 403
Error en la página 5: código 403
Error en la página 6: código 403
Error en la página 7: código 403
Error en la página 8: código 403
Error en la página 9: código 403
Error en la página 10: código 403
Error en la página 1: código 403
Error en la página 2: código 403
Error en la página 3: código 403
Error en la página 4: código 403
Error en la página 5: código 403
Error en la página 6: código 403
Error en la página 7: código 403
Error en la página 8: código 403
Error en la página 9: código 403
Error en la página 10: código 403
Empty DataFrame
Columns: []
Index: []


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Configurar el navegador sin interfaz gráfica (opcional)
options = Options()
options.add_argument("--headless")  # Ejecutar sin abrir ventana
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Iniciar navegador
driver = webdriver.Chrome(options=options)

# Ir a la página de búsqueda de coches Toyota
url = "https://ovoko.es/busqueda?q=toyota"
driver.get(url)

# Esperar a que cargue el contenido dinámico
time.sleep(5)

# Buscar los elementos con datos de coches
coches = driver.find_elements(By.CLASS_NAME, "ProductCard__Container-sc")  # ¡Aquí toca ajustar la clase exacta!

# Recolectar datos de cada coche
for coche in coches:
    try:
        titulo = coche.find_element(By.CLASS_NAME, "ProductCard__Title").text
        info_extra = coche.find_element(By.CLASS_NAME, "ProductCard__Attributes").text
        print("Título:", titulo)
        print("Año/KM u otros:", info_extra)
        print("-" * 30)
    except:
        continue

driver.quit()


- Ovoko usa precios en centavos, por eso dividimos por 100.
- Puedes ajustar el número de páginas (paginas=10) según cuántos repuestos quieras.
- Se pueden obtener otros campos como kilometraje, año del coche, o el enlace al producto si quieres.

Extraer nombre del repuesto, precio, marca (Toyota o Lexus), y quizás más detalles como referencia o estado del producto, si están disponibles.

🔍 1. Análisis inicial de la web
Al visitar https://ovoko.es, verás que:

Se trata de una Single Page Application (SPA) hecha en JavaScript.
El contenido (como el listado de productos) no aparece directamente en el HTML, sino que se carga dinámicamente usando JavaScript.
El scraping con requests + BeautifulSoup no funcionará bien aquí, porque no carga los datos directamente en el HTML.
🔁 Entonces… ¿qué hacemos?
Hay 2 caminos posibles:

Usar Selenium para simular un navegador y esperar a que el JS cargue los datos.
Buscar una API oculta que devuelva los datos en JSON (¡más limpio y rápido!).
Vamos a explorar las dos opciones.

🔎 2. ¿Tiene una API oculta? (¡Sí!)
Ovoko carga los resultados desde una API interna. Si inspeccionas la red (en las herramientas de desarrollador, pestaña "Network") al hacer una búsqueda, verás algo como:

https://api.ovoko.eu/es/v1/products/search?query=toyota&page=1&limit=24

✨ Esto es oro. Devuelve un JSON limpio con todos los datos del repuesto: nombre, precio, estado, marca, etc.

🧠 3. ¡Scraping limpio usando la API!
Aquí tienes un ejemplo completo en Python para consultar esta API.

# Obtener datos de Toyota y Lexus
df_toyota = get_ovoko_parts("toyota", paginas=10)
df_lexus = get_ovoko_parts("lexus", paginas=10)

# Unir ambos resultados
df_total = pd.concat([df_toyota, df_lexus], ignore_index=True)

# Guardar en CSV
df_total.to_csv("repuestos_toyota_lexus.csv", index=False, encoding="utf-8")

print(df_total.head())
📌 Detalles interesantes del scraping:
Ovoko usa precios en centavos, por eso dividimos por 100.
Puedes ajustar el número de páginas (paginas=10) según cuántos repuestos quieras.
Se pueden obtener otros campos como kilometraje, año del coche, o el enlace al producto si quieres.
📁 Resultado
Obtendrás un CSV con repuestos de Toyota y Lexus, como este:

Nombre	Precio (€)	Moneda	Marca	Ubicación	Estado	Referencia OEM
Sensor de temperatura motor	14.99	EUR	toyota	Lituania	Usado bueno	8942233010
Módulo ABS Lexus RX 400h	39.99	EUR	lexus	España	Usado bueno	440500E060
🧩 ¿Qué puedes hacer con estos datos?
Predecir precios con un modelo de regresión.
Analizar diferencias entre marcas o países.
Agrupar por tipo de pieza y estudiar su valor promedio.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Iniciar el navegador
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Opcional: sin abrir la ventana
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

# Resultados guardados aquí
resultados = []

# Parámetros
MARCAS_MODELOS = {
    "Toyota": ["Corolla", "Camry"],
    "Lexus": ["RX350", "IS250"]
}
ANOS = [2012, 2013]
PIEZAS = ["Engine", "Alternator", "Radiator", "ECM"]

for marca, modelos in MARCAS_MODELOS.items():
    for modelo in modelos:
        for ano in ANOS:
            for pieza in PIEZAS:
                print(f"Buscando {pieza} para {marca} {modelo} {ano}...")
                try:
                    driver.get("https://www.car-part.com/")

                    # Esperar al formulario y seleccionar valores
                    wait = WebDriverWait(driver, 10)

                    Select(driver.find_element(By.NAME, "userPart")).select_by_visible_text(pieza)
                    Select(driver.find_element(By.NAME, "userMake")).select_by_visible_text(marca)
                    Select(driver.find_element(By.NAME, "userModel")).select_by_visible_text(modelo)
                    Select(driver.find_element(By.NAME, "userYear")).select_by_visible_text(str(ano))

                    # Pulsar el botón
                    driver.find_element(By.NAME, "userSearch").click()

                    # Esperar a que cargue el iframe con resultados
                    wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))

                    # Esperar a que se carguen filas dentro del iframe
                    wait.until(EC.presence_of_element_located((By.TAG_NAME, "tr")))

                    # Leer las filas de la tabla
                    filas = driver.find_elements(By.TAG_NAME, "tr")

                    for fila in filas:
                        celdas = fila.find_elements(By.TAG_NAME, "td")
                        if len(celdas) >= 4:
                            pieza_text = celdas[0].text.strip()
                            info = celdas[1].text.strip()
                            precio = celdas[2].text.strip()
                            contacto = celdas[3].text.strip()
                            resultados.append({
                                "marca": marca,
                                "modelo": modelo,
                                "año": ano,
                                "pieza": pieza_text,
                                "info_extra": info,
                                "precio": precio,
                                "contacto": contacto
                            })

                    driver.switch_to.default_content()
                    time.sleep(2)

                except Exception as e:
                    print("Error:", e)
                    driver.switch_to.default_content()
                    continue

# Guardar resultados
df = pd.DataFrame(resultados)
df.to_csv("car_part_results.csv", index=False)
print("Datos guardados en car_part_results.csv")
driver.quit()